In [2]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp,  get_final_df
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [3]:
file_path = "../../../data/processed_data/acc_outer.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

features_acc_inner, category_acc_inner, category_names, df_acc_inner = get_final_df(data, is_acc_data=1, is_multilabel=1)

labels = category_acc_inner.copy()
features=features_acc_inner.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (1330, 3)
Labels Shape:  (1330, 4)


In [4]:
max_length = 500

for i in range (len(features)):
    if(len(features[i][0]<max_length)):
        features[i][0]=np.pad(features[i][0], (0,max_length-len(features[i][0])), 'constant', constant_values=(0))
        
    if(len(features[i][1]<max_length)):
        features[i][1]=np.pad(features[i][1], (0,max_length-len(features[i][1])), 'constant', constant_values=(0))
    
    if(len(features[i][2]<max_length)):
        features[i][2]=np.pad(features[i][2], (0,max_length-len(features[i][2])), 'constant', constant_values=(0))

In [5]:
feat= np.zeros(len(features)*3*max_length).reshape(-1, 3*max_length)
for i in range(len(features)):
    feat[i]= np.concatenate((features[i][0],features[i][1], features[i][2]), axis=None)
    

print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (1330, 1500)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [7]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1197, 1, 1500)
(1197, 4)
(133, 1, 1500)
(133, 4)


In [8]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [9]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#5: Acceleration Outer",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2023-10-09 20:24:11,372] A new study created in RDB with name: Case#5: Acceleration Outer


Epoch 10/50, Training Loss: 0.6793071269989014, Validation Loss: 0.6801747977733612
Epoch 20/50, Training Loss: 0.6691045165061951, Validation Loss: 0.6697633117437363
Epoch 30/50, Training Loss: 0.6516951282819112, Validation Loss: 0.6509859412908554
Epoch 40/50, Training Loss: 0.6236256678899129, Validation Loss: 0.6206526607275009


[I 2023-10-09 20:24:39,566] Trial 0 finished with value: 0.5751350373029709 and parameters: {'lr': 2.333178272268227e-05, 'dropout_prob': 0.25652954393502586, 'hidden_channels': 137, 'n_layers': 5, 'kernel_size': 5, 'stride': 6, 'padding': 8, 'pooling_type': 'max', 'use_batch_norm': True}. Best is trial 0 with value: 0.5751350373029709.


Epoch 50/50, Training Loss: 0.5800206899642945, Validation Loss: 0.5751350373029709
Epoch 10/50, Training Loss: 0.6292166272799175, Validation Loss: 0.6253595650196075
Epoch 20/50, Training Loss: 0.3214630643526713, Validation Loss: 0.3332318142056465
Epoch 30/50, Training Loss: 0.26593107879161837, Validation Loss: 0.3065815083682537
Epoch 40/50, Training Loss: 0.23665107091267903, Validation Loss: 0.33930128812789917


[I 2023-10-09 20:25:40,236] Trial 1 finished with value: 0.33166496455669403 and parameters: {'lr': 0.004185335379320031, 'dropout_prob': 0.45824680007669344, 'hidden_channels': 211, 'n_layers': 7, 'kernel_size': 3, 'stride': 3, 'padding': 9, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 1 with value: 0.33166496455669403.


...

Epoch 50/50, Training Loss: 0.2081575522820155, Validation Loss: 0.2650296315550804
Epoch 10/50, Training Loss: 0.43059112429618834, Validation Loss: 0.4227883294224739
Epoch 20/50, Training Loss: 0.29753495852152506, Validation Loss: 0.2891973853111267
Epoch 30/50, Training Loss: 0.2686532954374949, Validation Loss: 0.2697446644306183
Epoch 40/50, Training Loss: 0.24862361053625742, Validation Loss: 0.25102049484848976


[I 2023-10-09 22:07:58,059] Trial 98 finished with value: 0.25300003588199615 and parameters: {'lr': 7.647229245820717e-05, 'dropout_prob': 0.0735169944001114, 'hidden_channels': 171, 'n_layers': 7, 'kernel_size': 9, 'stride': 1, 'padding': 5, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 64 with value: 0.24092380702495575.


Epoch 50/50, Training Loss: 0.2246823052565257, Validation Loss: 0.25300003588199615
Epoch 10/50, Training Loss: 0.41839962601661684, Validation Loss: 0.4018450230360031
Epoch 20/50, Training Loss: 0.2877439608176549, Validation Loss: 0.30183907598257065
Epoch 30/50, Training Loss: 0.2654058873653412, Validation Loss: 0.2748390883207321
Epoch 40/50, Training Loss: 0.24969752430915831, Validation Loss: 0.2606152966618538


[I 2023-10-09 22:09:50,737] Trial 99 finished with value: 0.2615063861012459 and parameters: {'lr': 8.756432995334961e-05, 'dropout_prob': 0.07061394559538414, 'hidden_channels': 158, 'n_layers': 7, 'kernel_size': 9, 'stride': 1, 'padding': 5, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 64 with value: 0.24092380702495575.


Epoch 50/50, Training Loss: 0.24028387467066448, Validation Loss: 0.2615063861012459


In [10]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 9.932054721022286e-05, 'dropout_prob': 0.0789779531161162, 'hidden_channels': 207, 'n_layers': 4, 'kernel_size': 7, 'stride': 2, 'padding': 8, 'pooling_type': 'max', 'use_batch_norm': True}
Best final validation loss:
0.24092380702495575


In [11]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/acc_outer_best_configs.json"

# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

In [12]:
# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [14]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You